In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,54112
2,Huelva,Confirmados PDIA 14 días,4824
3,Huelva,Tasa PDIA 14 días,"940,0393631739969"
4,Huelva,Confirmados PDIA 7 días,2480
5,Huelva,Tasa PDIA 7 dias,"483,2706510513085"
6,Huelva,Total Confirmados,54321
7,Huelva,Curados,47383
8,Huelva,Fallecidos,426


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  54112.0


/tmp/ipykernel_22571/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  15328.0


/tmp/ipykernel_22571/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_22571/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_22571/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_22571/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 196 personas en los últimos 7 días 

Un positivo PCR cada 92 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,54112.0,4824.0,2480.0,513170.0,483.270651,940.039363,630.0
Huelva-Costa,31857.0,2787.0,1371.0,289548.0,473.496622,962.534709,318.0
Huelva (capital),15328.0,1567.0,735.0,143837.0,510.995085,1089.427616,175.0
Sierra de Huelva-Andévalo Central,5351.0,895.0,502.0,67391.0,744.906590,1328.070514,165.0
Condado-Campiña,16468.0,1135.0,604.0,156231.0,386.607011,726.488341,146.0
Valverde del Camino,1166.0,233.0,176.0,12750.0,1380.392157,1827.450980,54.0
Aljaraque,2034.0,252.0,123.0,21474.0,572.785694,1173.512154,32.0
Puebla de Guzmán,218.0,42.0,30.0,3092.0,970.245796,1358.344114,24.0
Almonte,2602.0,150.0,68.0,24507.0,277.471743,612.070021,20.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Silvestre de Guzmán,65.0,40.0,30.0,614.0,4885.993485,6514.657980,6.0
Puerto Moral,23.0,9.0,4.0,278.0,1438.848921,3237.410072,2.0
Fuenteheridos,42.0,22.0,10.0,709.0,1410.437236,3102.961918,3.0
Aracena,986.0,199.0,71.0,8255.0,860.084797,2410.660206,18.0
Galaroza,85.0,28.0,19.0,1382.0,1374.819103,2026.049204,7.0
Cumbres Mayores,203.0,33.0,13.0,1749.0,743.281875,1886.792453,1.0
Escacena del Campo,285.0,42.0,28.0,2287.0,1224.311325,1836.466987,5.0
Valverde del Camino,1166.0,233.0,176.0,12750.0,1380.392157,1827.450980,54.0
Corteconcepción,60.0,9.0,4.0,536.0,746.268657,1679.104478,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Almendro (El),65.0,4.0,1.0,840.0,119.047619,476.190476,0.0,0.250000
Alájar,45.0,10.0,3.0,759.0,395.256917,1317.523057,1.0,0.300000
Campillo (El),127.0,25.0,8.0,2024.0,395.256917,1235.177866,5.0,0.320000
Bollullos Par del Condado,1568.0,115.0,41.0,14387.0,284.979495,799.332731,6.0,0.356522
Aracena,986.0,199.0,71.0,8255.0,860.084797,2410.660206,18.0,0.356784
Villalba del Alcor,522.0,14.0,5.0,3366.0,148.544266,415.923945,0.0,0.357143
Almonaster la Real,125.0,27.0,10.0,1788.0,559.284116,1510.067114,4.0,0.370370
Lucena del Puerto,409.0,34.0,13.0,3261.0,398.650721,1042.624962,3.0,0.382353
Cumbres Mayores,203.0,33.0,13.0,1749.0,743.281875,1886.792453,1.0,0.393939
